# Curating data form crystal fields

In [1]:
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
from scipy.special import sph_harm as Ynm_num
# from sympy import poly, lambdify
from IPython.display import display, Math
# %config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from itertools import product
from sympy import I
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import HTML, display, Math, Latex
from matplotlib import pyplot as plt
%matplotlib widget
from pyperclip import copy as pypercopy

Reloading /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


## Class infrastructure for Ion, Bnm, Anm, CFparams

In [523]:
# class Ion(Atom):
#     '''
#     Same as an Atom, but with the added attribute of charge_state.
#     Also, the spectroscopic data is limited to that ion.
#     '''
#     def __init__(self, element, charge_state):
#         Atom.__init__(self,element)
#         self.charge_state = charge_state
#         self.nist_data = self.nist_data[self.nist_data['Charge'] == self.charge_state]
#         self.nist_data.reset_index(drop=True, inplace=True)

In [640]:
# class Bnm():
#     '''
#     A class for Bnm crystal field parameters.
    
#     Instantiated  with  a  dictionary that must contain at least
#     the following keys:
    
#     host            (str) e.g. 'MgO'
#     site            (str) e.g. 'Mg'
#     point_sym_group (str) e.g. 'C_{3v}'
#     ion         (str,int) e.g. ('Cr',3)
#     params         (dict) e.g. {(2,0): 1.34e3, (4,2): -2.34e3}
#     unit            (str) e.g. 'cm^{-1}'
#     sources        (list) e.g. ['Morrison 1982, Crystal F ...', ...]
#     comments       (list) e.g. ['This is very approximate.', 'Yup.']
#     experimental   (bool) e.g. False
    
#     Ideally  params should be accompanied by params_uncertainty,
#     a  dictionary with the same keys than params but with values
#     equal to corresponding uncertainties.
#     '''
#     def __init__(self, Bnm_params):
#         self.min_keys = set(['host','site','point_sym_group','params',
#                              'unit','sources', 'ion', 'is_experimental'])
#         self.energy_units = ['cm^{-1}','eV','J']
#         self.init_dict = Bnm_params
#         assert self.min_keys <= set(params.keys()), \
#             'provide at least: %s' % str(self.min_keys)
#         for attr_name in Bnm_params:
#             setattr(self, attr_name, Bnm_params[attr_name])
#         assert self.unit in self.energy_units, \
#                 "unit must be one of %s" % str(self.energy_units)
#     def __str__(self):
#         return '%s\n%s^%d+:%s\n%s' % (self.point_sym_group, self.ion[0], self.ion[1], self.host, str(self.params))
#     def __repr__(self):
#         return str(self.init_dict)

In [641]:
params = {
    'host':'MgO',
    'site':'Mg',
    'ion': ('Cr',3),
    'point_sym_group':'C_{4v}',
    'unit':'cm^{-1}',
    'sources':[],
    'params':{(1,0):1.1E-5,(2,2):1.1},
    'is_experimental': False
         }

In [516]:
# class Anm():
#     '''
#     A  class  for  Anm  crystal  field parameters. Initialized by giving a
#     dictionary  whose  keys  are  3-tuples  (n(int),m(int),t(str))  with t
#     either  'r'  (real),  'i' (imaginary), 'n' (norm) and whose values are
#     dictionaries with keys that must be a subset of Anm.AnmGoodKeys.

#     For this given dictionary of dictionaries, if the inner dictionary has
#     'total' as one key, then self.totalAnm is simply a dictionary keyed by
#     2-tuples  (n,m)  and  valued by the 'total' value; if 'total' is not a
#     key  then  all of the inner dictionaries are added up as a function of
#     (n,m).  At  this  stage  whether  the  quantity refered to is real, or
#     imaginary  has  been  included  as  the  third  element of the 3-tuple
#     (n,m,t),  but when the total is computed then this goes along with the
#     value.

#     The 'unit' key in the params dictionaries is used to infer the unit of
#     lenght being used, and the unit of energy being considered.

#     --------
#     Examples
    
#     params = {(2,0,r): {'mono': 1.34e-6, 'dipole': 1.25e-5, 'unit': 'cm^{-1}/Å^{2}'},
#               (4,4,i): {'mono': 4.55e-6, 'dipole': -1.23e-6, 'unit': 'cm^{-1}/Å^{4}'}}}
#     Anm({'params': params})
    
#     '''
#     Anmkeys = [('params',dict), # a dictionary of dictionaries
#                ('refs',list)]
#     AnmGoodKeys = ['mono','self-i','dipole','total', 'unit'] # this are the allowed keys for a specific Anm
#     def __init__(self, Anmdict):
#         for k in self.Anmkeys:
#             attr_name, attr_type = k
#             if attr_name in Anmdict:
#                 assert isinstance(Anmdict[attr_name],k[1]), '%s key must be of %s or not given' % (attr_name, attr_type)
#                 setattr(self, attr_name, Anmdict[attr_name])
#             else:
#                 setattr(self, attr_name, None)
#         if self.params:
#             self.totalAnm = {}
#             for key, value_dict in self.params.items():
#                 assert set(list(value_dict)).issubset(self.AnmGoodKeys)
#                 n, m, t = key
#                 unit = value_dict.pop('unit')
#                 if 'total' in value_dict.keys():
#                     self.totalAnm[(n,m)] = value_dict['total']
#                 else:
#                     self.totalAnm[(n,m)] = sum(list(value_dict.values()))
#                 if t == 'i':
#                     self.totalAnm[(n,m)] = self.totalAnm[(n,m)]*1j
#                 elif t in['r', 'n']:
#                     pass
#             self.totalAnm_length_unit = unit.split('/')[-1].split('^')[0]
#             self.totalAnm_energy_unit = unit.split('/')[0]
#     def Bnm_compute(self, ion):
#         '''
#         This  function  collects  the  relevant  Hartree-Fock radial
#         averages  and  approximates  the corresponding Bnm using the
#         totalAnm parameters.

#         It returns a Bnm object.
#         '''
#         Bnmdict = {}
#         Bnmparams = {}
#         assert self.totalAnm_length_unit == 'Å', 'Hartree Fock data requires unit of lenght to be Å.'
#         for k,v in self.totalAnm.items():
#             Bnmparams[k] = v * HartreeFockData().radial_average(ion.atomic_number,ion.charge_state,k[0])
#         Bnmdict['params'] = Bnmparams
#         Bnmdict['unit'] = self.totalAnm_energy_unit
#         return Bnm(Bnmdict)    

# class CFparams():
#     cfkeys = [('crystal_str', str),
#               ('space_group_str',str),
#               ('crystal_site', tuple),
#               ('Bnm', Bnm),
#               ('comments',list)]
#     def __init__(self, cfpdict):
#         for k in self.cfkeys:
#             attr_name, attr_type = k
#             if attr_name in cfpdict:
#                 assert isinstance(cfpdict[attr_name],k[1]), '%s key must be of %s or not given' % (attr_name, attr_type)
#                 setattr(self, attr_name, cfpdict[attr_name])
#             else:
#                 setattr(self, attr_name, None)

## Parsing some of the tables from Morrison

In [20]:
# loader for the data in Morrison's book
# this sheet is an export of the data on 
# a Google Spreadsheet "Morrison Data"
morrison_sheets = pd.read_excel('/Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',None)

In [83]:
group_label_to_index = {gl:(1+CPGs.all_group_labels.index(gl)) for gl in CPGs.all_group_labels}
index_to_group_label = {(1+CPGs.all_group_labels.index(gl)):gl for gl in CPGs.all_group_labels}
morrison = {}
# parse data for the free-ion model
frows = ['nd^N','Ion','Charge','F^{(2)}','F^{(4)}','\\zeta','\\alpha','Reference']
morrison['Free-ion params'] = pd.concat([morrison_sheets['1.'+x][frows] for x in 'A B C'.split(' ')])
morrison['Free-ion params']['Charge'] = morrison['Free-ion params']['Charge'].astype(int)

In [93]:
# parse data for number of Bnm and Anm as a function of group number
# How many Bnms or Anms for the d^N configuration
# this will just be a dictionary
BnmAnms = morrison_sheets['1.D'][['Group No.','n=1','n=2','n=3','n=4','n=5']]
grup_labels = (BnmAnms['Group No.'].apply(lambda x: index_to_group_label[int(x)]))
BnmAnms.loc[:,"Group Label"] = grup_labels
for i in [1,2,3,4,5]:
    BnmAnms.loc[:,'n=%d' % i] = BnmAnms['n=%d' % i].astype(int)
BnmAnms.loc[:,'Group No.'] = BnmAnms['Group No.'].astype(int)
morrison['BnmAnms'] = BnmAnms
# parse data for Hartree-Fock
hartree = morrison_sheets['2']
metadat = list(morrison_sheets['2'])[0]
hartree = hartree[list(hartree)[1::]]
hartree.attrs['metadata'] = metadat
hartree['N'] = hartree['nd^N'].apply(lambda x: int(x.split('d')[-1]))
hartree['n'] = hartree['nd^N'].apply(lambda x: int(x.split('d')[0]))
hartree['Charge'] = hartree['Charge'].astype(int)
hartree['Z'] = hartree['Z'].astype(int)
hartree.drop(columns=['nd^N'], inplace=True)
morrison['Hartree-Fock'] = hartree

In [94]:
for x in 'A B C D'.split(' '):
    print(list(morrison_sheets['1.'+x]))

['Free-ion data: F^{(2)}, F^{(4)}, \\zeta, and \\alpha for nd^N ions (cm^-1) (Α) 3δ^ν', 'nd^N', 'Ion', 'Charge', 'F^{(2)}', 'F^{(4)}', '\\zeta', '\\alpha', 'Reference', 'Comments', 'Types']
['Free-ion data: F^{(2)}, F^{(4)}, \\zeta, and \\alpha for nd^N ions (cm^-1) (B) 4d^N', 'nd^N', 'Ion', 'Charge', 'F^{(2)}', 'F^{(4)}', '\\zeta', '\\alpha', 'Reference', 'Comments', 'Types']
['Free-ion data: F^2, F^4, ζ, and α for nd^N ions (cm^-1) (C) 5d^N', 'nd^N', 'Ion', 'Charge', 'F^{(2)}', 'F^{(4)}', '\\zeta', '\\alpha', 'Reference', 'Comments', 'Types']
['Table 1D. Number of Bnm or Anm for the d^N configuration for 30 of 32 point groups', 'Group No.', 'n=1', 'n=2', 'n=3', 'n=4', 'n=5', 'Host', 'Site', 'Comments', 'Types']


In [7]:
allBees = []
for n in [1,2,3,4,5,6]:
    for m in range(0,n+1):
        allBees.append('B%d%d.re' % (n,m))
        allBees.append('B%d%d.im' % (n,m))
t8p1v1 = ('\n'.join(morrison_sheets['AngMom8.1and8.2']['8p1'][1:]))
header = morrison_sheets['AngMom8.1and8.2']['8p1'][0]
rows = []
for idx, line in zip(range(3,16),t8p1v1.split('\n')):
    rows.append({what:(x=='x') for what,x in zip(header.split(),line)})
full_params = {}
for group_num, row in zip(range(3,16),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
t8p2v1 = ('\n'.join(morrison_sheets['AngMom8.1and8.2']['8p2'][1:-1]))
header = morrison_sheets['AngMom8.1and8.2']['8p2'][0]
rows = []
for line in t8p2v1.split('\n'):
    rows.append({what:(x=='x') for what,x in zip(header.split(),line)})
for group_num, row in zip(range(16,28),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
rows = [
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True}
]
for group_num, row in zip(range(28,33),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
def fixer(s):
    see, kind = s.split('.')
    sign = ''
    if '-' in see:
        sign = '-'
        see = see.replace('-','')
    better = '%s_{%s,%s}^%s' % (see[0],see[1:][0],sign+see[1:][1],kind)
    better = better.replace('im','i').replace('re','r')
    return better
full_params = {group_num : {sp.Symbol(fixer(k)): v for k,v in full_params[group_num].items()} for group_num in full_params}
morrison['Bkq grid from tables 8.1-8.3 in_Morrison 1988'] = full_params

In [8]:
cf_components_keys = [k for k in morrison_sheets if 'crystal-field components' in list(morrison_sheets[k])[0].lower()]

In [9]:
crystallographic_sheets = {k.split('.')[0]: list(morrison_sheets[k])[0].split(' on')[-1] for k in morrison_sheets if 'Crystallographic Data' in list(morrison_sheets[k])[0]}

In [11]:
for cf_key in cf_components_keys:
    print(cf_key)
    component_data = morrison_sheets[cf_key]
    pivot = list(component_data)[0]
    for_what = (pivot.split('for ')[-1])
    print(for_what)
    subs = for_what.split(' ')[0]
    print(subs)

3.2.1
Al2 (S4) site
Al2
3.3.1
Al1 (C3i) site (rotated so that z-axis is parallel to (111) crystallographic axis)
Al1
20.2.1
Zn (T_d) site of cubic ZnS
Zn
20.2.2
Zn (C3v) site of hexagonal ZnS
Zn
20.2.3
Zn (C3v) site of hexagonal ZnS (Kisi and Elcombe, 1989)
Zn


In [12]:
special_reps = {}
# in all cases 
generic_reps = {sp.Symbol(fixer('B%d%d.re'%(k,q))): (-sp.S(1))**q*sp.Symbol(fixer('B%d%d.re'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) }
generic_reps.update({sp.Symbol(fixer('B%d%d.im'%(k,q))): -(-sp.S(1))**q*sp.Symbol(fixer('B%d%d.im'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) })
# in some groups some params are related between one another
special_reps = {i:generic_reps for i in range(1,28)}
treps = {sp.Symbol(fixer('B44.re')): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol(fixer('B40.re')),
        sp.Symbol(fixer('B64.re')): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol(fixer('B60.re'))}
treps.update(special_reps)
special_reps[28] = treps
special_reps[29] = treps
special_reps[30] = treps
special_reps[31] = treps
special_reps[32] = treps
cnm_reps = {sp.Symbol('C_{%d,%d}'%(k,q)): (sp.S(-1))**q*sp.conjugate(sp.Symbol('C_{%d,%d}'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) }
morrison['special_reps'] = special_reps

In [13]:
crystal_fields = {}
for i in range(1,33):
    crystal_fields[i] = compute_crystal_field(i)

Too little symmetry, returning empty list.
Too little symmetry, returning empty list.


In [44]:
full_params = morrison['Bkq grid from tables 8.1-8.3 in_Morrison 1988']

In [15]:
# this  bit of code is necessary to make a
# raw  (just  dictionaries) of the crystal
# fields, I know, it's a bit awkward.
crystal_fields_raw = {}
for k in crystal_fields:
    if k in [1,2]:
        crystal_fields_raw[k] = []
    else:
        crystal_fields_raw[k] = [q.dict for q in crystal_fields[k]]
morrison['crystal_fields_raw'] = crystal_fields_raw       

In [17]:
if input("continue?") == 'yes':
    pickle.dump(morrison,open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/morrison.pkl','wb'))

In [95]:
# eigenbasis = eigensys[1][1][2]
# fromvectoqet(mbasis(2,True),eigenbasis[1])

## Making symbolic expression for crystal fields in different symmetries

In [102]:
def irtransform(eigenbasis, l, group_num):
    # given a basis eigenbasis for a subspace for a fixed value of l
    # and the index group_num for a point group
    # return a dictionary whose keys are the 
    # irreducible representation of the group
    # and whose values are dictionaries whose
    # keys are labels for group basis vectors
    # and whose values are linear combinations
    # of the elements in lbasis
    # the lbasis is as the set of coefficients
    # for the ordered basis provided by mbasis(l)
    group = CPGs.groups[group_num]

In [103]:
def fromvectoqet(orderedbasis,coeffvec):
    '''
    Given a vector of coefficients and 
    its corresponding ordered basis,
    return what would be the corresponding
    Qet.
    '''
    return Qet({ovec: coeff for ovec, coeff in zip(orderedbasis, coeffvec)})

In [104]:
def eigen_latex(eigensys, l, normalized=False):
    eigen_chunks = []
    mbase = mbasis(l)
    for eigen_chunk in eigensys:
        eigenval_latex = sp.latex(Bsimple(eigen_chunk[0]))
        if normalized:
            eigenvecs_latex = ', '.join(([sp.latex((mbase * eigenvec.normalized())[0]) for eigenvec in eigen_chunk[2]]))
        else:
            eigenvecs_latex = ', '.join(([sp.latex((mbase * eigenvec)[0]) for eigenvec in eigen_chunk[2]]))
        eigen_chunks.append('%s: \{{%s}\}' % (eigenval_latex, eigenvecs_latex))
    eigen_final = '\]\\\\ \n\['.join(eigen_chunks)
    eigen_final = '\[' + eigen_final + '\]'
    return eigen_final

In [105]:
def mbasis(l, astuples=False):
    '''
    Return a row matrix with symbols corresponding to the kets
    that span the angular momentum space for a given value of l.
    '''
    if astuples:
        return sp.Matrix([[(l,ml) for ml in range(-l,l+1)]])
    else:
        return sp.Matrix([[sp.Symbol('|%d,%d\\rangle' % (l,ml)) for ml in range(-l,l+1)]])

In [106]:
diagonalley = {}
l = 2
for group_num in range(1,33):
    print(group_num, end=',')
    cf = CrystalField(group_num)
    cf.matrix_rep_symb(l)
    if group_num in range(3,9):
        print("Symbolic diagonalization known to fail.")
        eigensys = []
    else:
        try:
            eigensys = cf.splitter(l)
        except:
            print("Failed in symbolic diagonalization.")
            eigensys = []
    summa = {}
    summa['simple_hams'] = cf.simplified_ham
    summa['simple_hams_latex'] = list(map(sp.latex,cf.simplified_ham))
    summa['matrices'] = cf.matrix_rep_symb(l)
    summa['splitting.latex'] = sp.latex(l_splitter(group_num, l).as_symbol_sum())
    summa['simple_matrices'] = [Bsimple(mat) for mat in cf.matrix_rep_symb(l)]
    summa['simple_matrices.latex'] = [sp.latex(smat) for smat in summa['simple_matrices']]
    summa['free_parameters'] = [mat.free_symbols for mat in summa['matrices']]
    summa['simple_free_parameters'] = [mat.free_symbols for mat in summa['simple_matrices']]
    summa['simple_free_parameters.latex'] = list(map(sp.latex,summa['simple_free_parameters']))
    summa['eigen_system'] = eigensys
    summa['eigen_latex'] = [eigen_latex(esys, l) for esys in eigensys]
    summa['group_num'] = group_num
    summa['group_symb'] = CPGs.groups[group_num].label
    diagonalley[group_num] = summa

1,2,3,Symbolic diagonalization known to fail.
4,Symbolic diagonalization known to fail.
5,Symbolic diagonalization known to fail.
6,Symbolic diagonalization known to fail.
7,Symbolic diagonalization known to fail.
8,Symbolic diagonalization known to fail.
9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,

In [113]:
selector = {'simple_matrices':'matrices', 'simple_free_parameters':'simple_free_parameters','free_parameters':'free_parameters','eigen_system':'eigen_system'}
crystal_pickle = {i : {selector[s]: diagonalley[i][s]  for s in selector} for i in diagonalley}
pickle.dump(crystal_pickle, open('/Users/juan/Google Drive/Zia Lab/Log/Data/crystal_splits.pkl','wb'))

In [108]:
templates = (
{'header': r'''{{\vspace{{0.5cm}}
\Large{{------------ ${group_symb}$ ({group_num}) ------------}}}}'''+'\\\\ \n',
'body0':''' \\vspace{{0.5cm}}

$V_{{CF}} = {simple_ham}$ \\\\

\[V_{{CF}}^{{l=2}}={matrix_latex}\]

\\vspace{{0.25cm}}
${splitting_latex}$ \\\\
\\vspace{{0.5cm}}
Free-parameters for d-electrons: ${simple_free_parameters}$ \\\\
\\vspace{{0.5cm}}
Eigenvalues and eigenvectors:

{eigen_latex} \n
\\vspace{{0.25cm}}
''',
'body1':''' \\vspace{{0.5cm}}
$V_{{CF}} = {simple_ham}$ \\\\
\[V_{{CF}}^{{l=2}}={matrix_latex}\]

${splitting_latex}$ \\\\
\\vspace{{0.5cm}}
Free-parameters for d-electrons: ${simple_free_parameters}$ \\\\
\\vspace{{0.25cm}}
'''})
full_latex = []
double_col_entries = [6,21]
double_col_exits = [15,32]
for group_num in diagonalley:
    summary = diagonalley[group_num]
    latex_output = [templates['header'].format(**summary)]
    if len(summary['eigen_latex']) > 0:
        for simple_ham, mat_lat, free_params, eigen_tex in zip(summary['simple_hams_latex'],
                                                   summary['simple_matrices.latex'],
                                                   summary['simple_free_parameters.latex'],
                                                   summary['eigen_latex']):
            out = templates['body0'].format(**{'simple_ham': simple_ham,
                         'simple_free_parameters': free_params, 
                         'matrix_latex': mat_lat,
                         'eigen_latex': eigen_tex,
                         'splitting_latex': summary['splitting.latex'].replace('+','\\oplus')})
            latex_output.append(out)
    else:
        for simple_ham, mat_lat, free_params, splittex in zip(summary['simple_hams_latex'],
                                                    summary['simple_matrices.latex'],
                                                    summary['simple_free_parameters.latex'],
                                                    summary['splitting.latex']):
            out = templates['body1'].format(**{'simple_ham': simple_ham,
                         'simple_free_parameters': free_params, 
                         'matrix_latex': mat_lat,
                         'splitting_latex': summary['splitting.latex'].replace('+','\\oplus')})

            latex_output.append(out)
    final_output = '\\begin{center}\n\n%s\n\\end{center}\n \\hrule \n' % (' '.join(latex_output))
    if group_num in double_col_entries:
        final_output = '\n\\begin{multicols}{2}\n' + final_output
    if group_num in double_col_exits:
        final_output = final_output + '\n\\end{multicols}\n'
    full_latex.append(final_output)
final_latex = '\n'.join(full_latex)
# pyperclip.copy(final_latex)
fname = '/Users/juan/Google Drive/Zia Lab/Theoretical Division/1delectron.tex'
open(fname,'w').write(final_latex)

48049

In [432]:
# latex_printout = {}
# for group_num in diagonalley:
#     this_chunk = diagonalley[group_num]
#     group_label = CPGs.groups[group_num].label
#     matrix_latex = []
#     for matrix in this_chunk['matrices']:
#         matrix_latex.append(sp.latex(matrix))
#     matrix_latex = '\n'.join(matrix_latex)

## Parsing index of book to create connection between section number and chemical formula of hosts.

End result of this is that an additional key is added to the "morrison" dictionary
whose keys are the section numbers of the book and whose values are the parsed formulas
for the different crystal hosts.

In [125]:
import re

In [115]:
def simple_formula_parser(job):
    '''
    Parse a chemical formula into a dictionary of
    species strings and quantities.

    Formula cannot contain parenthesis.
    '''
    sform = job[0]
    mult = job[1]
    formula = {}
    formula['original'] = sform
    sform = re.sub(r'([1-9]+)',r' \1 ',sform)
    sform = re.sub(r'([a-z])([A-Z])','\\1 1 \\2',sform)
    sform = re.sub(r'([A-Z])([A-Z])','\\1 1 \\2',sform)
    if sform[-1] not in '1 2 3 4 5 6 7 8 9'.split(' '):
        sform = sform+' 1'
    sform = sform.strip()
    parsed = sform.split(' ')
    parsed = dict(list(zip(parsed[::2],list(map(lambda x: int(x)*mult,parsed[1::2])))))
    formula['parsed'] = parsed
    return {k:v for k,v in parsed.items() if k !=''}

def complex_formula_parser(sform):
    '''
    Parse  a  chemical  formula  into  a list of dictionary that
    specify elements in keys and number of atoms as values.

    The  different dictionaries relate to the different parts of
    the formula as parsed from the parenthesis structure.
    
    -------
    Example
    
    complex_formula_parser(Ca(CO2)2)   returns  a  list  of  two
    dictionaries  one  for  the single calcium atom, and another
    for the oxalate part of the formula like so:
    
       --> [{'C': 2, 'O': 4}, {'Ca': 1}]
    '''

    if sform[-1] == ')':
        sform = sform + '1'
    if ')' in sform:
        multipliers = list(map(int,re.findall('\)(.)',sform)))
        groups = list(zip(re.findall(r'\((.*?)\)',sform), multipliers))
        # delete every single digit after a right parenthesis
        sform = re.sub(r'\)[0-9]',r')',sform)
        # delete every thing inside parentheses
        sform = re.sub(r'\(.+\)','',sform)
        groups = groups + [(sform,1)]
    else:
        groups = [(sform,1)]
    return list(map(simple_formula_parser,groups))

In [428]:
morrison_sheets = pd.read_excel('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',None)

In [121]:
index_df = morrison_sheets['Index']
index_df.loc[:,'Section'] = index_df['Section'].astype(int)
# some are combinations of this and that, I shall omit these
# i am keeping the ones in which there are single substitutions of "X"
index_df = index_df[index_df['Comment1'].apply(lambda x: '*' not in str(x))]
index_df.loc[:,'Parsed Formula'] = index_df['Formula'].apply(complex_formula_parser)
index_df.loc[:,'Pretty Name'] = index_df['Pretty Name'].fillna('')

In [123]:
index = dict(zip(index_df['Section'],zip(index_df['Formula'],index_df['Parsed Formula'],index_df['Pretty Name'])))
morrison['section_to_host'] = index

In [390]:
if input("continue?") == 'yes':
    pickle.dump(morrison,open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/morrison.pkl','wb'))

## Parsing tables from Morrisons book

In [169]:
morrison_sheets = pd.read_excel('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',None)

In [170]:
morrison_keys = list(morrison_sheets)

In [4]:
def parse_comments(dframe):
    '''
    Receive a table an result a list of strings with the comments in the
    comments column.
    '''
    if 'Comments' in dframe:
        return list(filter(lambda x: x != '',list(dframe['Comments'].fillna(''))))
    else:
        print("No comments column, returning empty.")
        return []

In [6]:
def table_types(dframe):
    '''
    Receive a table and return a list with the types of thing it is.
    '''
    if 'Types' in list(dframe):
        return dframe['Types'][0].split(',')
    else:
        print("No types, returning empty.")
        return []

In [7]:
def parse_table(dframe, debug=False):
    # get the type of table it is
    dframe_types = table_types(dframe)
    if ('CDATA' in dframe_types):
        if debug:
            print("This is a table with crystallographic data.")
        # lattice params are in the first column
        crystallographic_data = list(dframe)[0]
        first_col = [s for s in dframe[list(dframe)[0]].fillna('') if s]
        lattice_params = {s.split('=')[0]:float(s.split('=')[1]) for s in first_col}
        crystal_system = crystallographic_data.split(' ')[0].lower()
        if ('GARNET') in dframe_types:
            print("This is a table with an array of garnets.")
        else:
            pass
        crystal_data = {'lattice_params': lattice_params,
                       'crystal_system': crystal_system}
        return crystal_data
    if ('EXPARAMS' in dframe_types):
        pass

In [174]:
# collect sheets that have crystallographic data
cdata_sheets = {mk:table_types(morrison_sheets[mk]) for mk in morrison_keys}
cdata_sheets = [k for k,v in cdata_sheets.items() if 'CDATA' in v]
for cs in cdata_sheets:
    print(cs,parse_table(morrison_sheets[cs]))

3.1 {'lattice_params': {'a': 12.0}, 'crystal_system': 'cubic'}
4.1.1 {'lattice_params': {'a': 8.11}, 'crystal_system': 'cubic'}
5.1.1 {'lattice_params': {'a ': 9.96}, 'crystal_system': 'cubic'}
5.1.2 {'lattice_params': {'a ': 6.15, 'c ': 4.96}, 'crystal_system': 'hexagonal'}
6.1 {'lattice_params': {'a': 4.724, 'c': 8.225}, 'crystal_system': 'trigonal'}
7.1 {'lattice_params': {'a': 4.623, 'b': 3.052}, 'crystal_system': 'tetragonal'}
8.1 {'lattice_params': {'a': 4.8734, 'c': 3.3099}, 'crystal_system': 'tetragonal'}
9.1 {'lattice_params': {'a': 4.7034, 'c': 3.1335}, 'crystal_system': 'tetragonal'}
10.1 {'lattice_params': {'a': 4.2112}, 'crystal_system': 'cubic'}
11.1 {'lattice_params': {'a': 9.206, 'c': 9.205}, 'crystal_system': 'hexagonal'}
12.1 {'lattice_params': {'a': 12.415}, 'crystal_system': 'cubic'}
13.1 {'lattice_params': {'a': 10.81}, 'crystal_system': 'cubic'}
20.1.1 {'lattice_params': {'a': 5.4093}, 'crystal_system': 'cubic'}
20.1.2 {'lattice_params': {'a': 3.8227, 'b': 6.2607}

## Parsing data from sheets with Experimental Values for CF params

In [777]:
morrison_sheets = pd.read_excel('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',None)

In [778]:
morrison_keys = list(morrison_sheets.keys())

In [779]:
exparams_sheets = {mk:table_types(morrison_sheets[mk]) for mk in morrison_keys}
exparams_sheets = [k for k,v in exparams_sheets.items() if ('EXPARAMS' in v and len(v) == 1)]
' || '.join(exparams_sheets)

'3.4 || 5.3.3 || 6.3 || 7.3 || 8.3 || 9.3 || 10.3 || 11.3 || 13.3 || 20.3'

In [780]:
index = morrison['section_to_host'] 

In [781]:
# these sheets provide information on experimentally
# measured values for F^2, F^4, zeta, and B_40
# the result should be something that is keyed
# by the formulas of the hosts
# and whose values are dataframes
dframes = []
for sheetkey in exparams_sheets:
    # parse the section for the sheet key
    section = int(sheetkey.split('.')[0])
    # query to which host this refers to
    host = index[section][0]
    # grab the sheet
    asheet = pd.DataFrame(morrison_sheets[sheetkey])
    asheet.loc[:,"Host"] = host
    asheet.loc[:,"Type"] = 'Experimental'
    asheet.loc[:,"Source"] = 'Morrison - 1992 - Crystal Fields for Transition-Metal Ions in Lasers: %s' % sheetkey
    # grab the metadata which is in the top left corner
    metadata = list(asheet)[0]
    if 'site' in metadata:
        print(metadata)
#     asheet.loc[:,"metadata"] = metadata
    asheet = asheet[list(asheet)[1:]]
    ion = asheet['Ion']
    
    # drop the comments section
    asheet.drop(['Comments'],axis=1,inplace=True)
    # drop the first column
#     asheet.drop(asheet.columns[0],axis=1,inplace=True)
    asheet.reset_index(drop=True,inplace=True)
    # drop the types column
    asheet.drop(['Types'],axis=1,inplace=True)
    dframes.append(asheet)

Experimental values (cm^{-1}) of parameters for D_{3d} site of Cs2TiF6


In [782]:
for idx, df in enumerate(dframes):
    if 'Reference' in list(df):
        print(exparams_sheets[idx],list(df))

In [783]:
for idx, dframe in enumerate(dframes):
    if 'Site' not in dframe.columns:
        print("no site",exparams_sheets[idx])
    else:
        print("site",exparams_sheets[idx])

site 3.4
site 5.3.3
site 6.3
site 7.3
site 8.3
site 9.3
site 10.3
site 11.3
site 13.3
site 20.3


In [784]:
Bnm_df = pd.concat(dframes)
Bnm_df.reset_index(drop=True,inplace=True)

In [785]:
list(Bnm_df[Bnm_df['nd^N'].apply(lambda x: 'd2' in x)]['Source'])

['Morrison - 1992 - Crystal Fields for Transition-Metal Ions in Lasers: 3.4',
 'Morrison - 1992 - Crystal Fields for Transition-Metal Ions in Lasers: 3.4',
 'Morrison - 1992 - Crystal Fields for Transition-Metal Ions in Lasers: 11.3']

In [786]:
Bnm_df.to_pickle(os.path.join(module_dir,'data','CFparams.pkl'))

In [787]:
Bnm_df.to_excel(os.path.join(module_dir,'data','CFparams.xlsx'))


In [788]:
!ding

In [789]:
ions = set(tot_df['Ion'])
counting_df = pd.DataFrame([[ion,list(tot_df['Ion']).count(ion),', '.join(list(tot_df[tot_df['Ion']==ion]["Host"]))] for ion in ions],columns=['Ion','Count','Hosts']).sort_values('Count')
counting_df.to_clipboard(excel=False)

In [790]:
print("Hosts:")
print("---",' | '.join(list(set(tot_df['Host']))))
print("Point Groups:")
print("---",' | '.join(list(set(tot_df['Symmetry']))))
print("Ions:")
print("---",' | '.join(list(ions)))

Hosts:
--- MgF2 | ZnF2 | MnF2 | Cs2TiF6 | ZnS | Be3Al2(SiO3)6 | Cs2SnBr6 | Y3Al5O12 | NH4Al(SO4)2 | MgO
Point Groups:
--- C_{3i} | S_{4} | D_{3} | D_{2h} | D_{3d} | T_{d} | O_{h}
Ions:
--- Mn | Rh | V | Ni | Pd | Os | Pt | Cu | Fe | Co | Cr


In [378]:
import pickle5

In [382]:
ferrenti_df = pickle5.load(open('/Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/ferrenti_gt.pkl','rb'))

In [387]:
morrison_hosts = set(map(lambda x: x[0],index.values()))
ferrenti_hosts = set(ferrenti_df['pretty_formula'])

In [460]:
intersection = list(ferrenti_hosts.intersection(morrison_hosts))

In [481]:
tot_df[tot_df['Host'].apply(lambda x: x in intersection)]

,Ion,Charge,nd^N,F^{(2)},F^{(4)},"B_{4,0}",T (K),Site,Symmetry,Ref,Host,\zeta,"B_{2,0}","B_{4,3}",\alpha,"B_{4,4}","B_{2,2}","B_{4,2}"
43,V,2,3d3,42429.00,30239.0,30429,NaN,Mg,O_{h},59,MgO,NaN,NaN,NaN,60.0,NaN,NaN,NaN
44,V,2,3d3,47625.00,25455.0,29400,NaN,Mg,O_{h},55,MgO,NaN,NaN,NaN,79.0,NaN,NaN,NaN
45,V,2,3d3,44275.00,31185.0,27720,NaN,Mg,O_{h},56,MgO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Cr,3,3d3,50906.00,37825.0,33579,NaN,Mg,O_{h},59,MgO,NaN,NaN,NaN,70.0,NaN,NaN,NaN
47,Cr,3,3d3,52780.00,36792.0,33390,NaN,Mg,O_{h},7,MgO,270.0,NaN,NaN,70.0,NaN,NaN,NaN
48,Cr,3,3d3,54600.00,40950.0,34020,NaN,Mg,O_{h},28,MgO,135.0,NaN,NaN,NaN,NaN,NaN,NaN
49,Cr,3,3d3,54250.00,40320.0,34860,NaN,Mg,O_{h},36,MgO,210.0,NaN,NaN,NaN,NaN,NaN,NaN
50,Cr,2,3d4,NaN,NaN,14007,NaN,Mg,O_{h},12,MgO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Mn,2,3d5,60984.00,40446.0,20559,NaN,Mg,O_{h},11,MgO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Mn,2,3d5,67550.00,41240.0,20580,NaN,Mg,O_{h},23,MgO,320.0,NaN,NaN,65.0,NaN,NaN,NaN


In [654]:
# nist_levels = pd.read_pickle('/Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/nist_atomic_spectra_database_levels.pkl')

In [494]:
lanthanides = 'La Ce Pr Nd Pm Sm Eu Gd Tb Dy Ho Er Tm Yb Lu'.split(' ')

In [499]:
for lanthanide in lanthanides:
    at = Atom(lanthanide)
    ndata = at.nist_data

## Parsing data from sheets with Theoretical Values for CF params

In [673]:
# morrison_sheets = pd.read_excel('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',['3.3.2'])
# morrison_keys = list(morrison_sheets.keys())

In [2]:
d_sheets = ['3.3.2','4.2.2','5.2.2','5.3.2','10.2.2','11.2.2','12.3.2','12.3.3','12.3.4','38.2.3']
morrison_sheets = pd.read_excel('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/Morrison.xlsx',d_sheets)
morrison_keys = list(morrison_sheets.keys())

In [793]:
for mk in morrison_keys:
    print(mk)
    print(table_types(morrison_sheets[mk]))

3.3.2
['THEOCFCOMP']
4.2.2
['THEOCFCOMP']
5.2.2
['THEOCFCOMP']
5.3.2
['THEOCFCOMP']
10.2.2
['THEOCFCOMP']
11.2.2
['THEOCFCOMP']
12.3.2
['THEOCFCOMP']
12.3.3
['THEOCFCOMP']
12.3.4
['THEOCFCOMP']
38.2.3
['THEOCFCOMP']


In [8]:
theoparams_sheets = {mk:table_types(morrison_sheets[mk]) for mk in morrison_keys}
theoparams_sheets = [k for k,v in theoparams_sheets.items() if ('THEOCFCOMP' in v and len(v) == 1)]
'THEOCFCOMP: ' + (' || '.join(theoparams_sheets))

'THEOCFCOMP: 3.3.2 || 4.2.2 || 5.2.2 || 5.3.2 || 10.2.2 || 11.2.2 || 12.3.2 || 12.3.3 || 12.3.4 || 38.2.3'

In [9]:
index = morrison['section_to_host'] 

In [13]:
def listin(lista, string):
    '''
    In a list of strings determine if the given string is in any
    of them.
    This is case insensitive.
    '''
    return sum(list(map(lambda x: string.lower() in x.lower(), lista))) > 0
def listreplace(lista, find_and_replace):
    '''
    Takes a list of string and a tuple (find, replace)
    that replaces every occurence of replace by find in every 
    string of the list
    '''
    return [s.replace(*find_and_replace) for s in lista]

In [15]:
dframes = []
for sheetkey in theoparams_sheets:
    # parse the section for the sheet key
    section = int(sheetkey.split('.')[0])
    # query to which host this refers to
    host = index[section][0]
    # grab the sheet
    asheet = pd.DataFrame(morrison_sheets[sheetkey])
    asheet.loc[:,"Host"] = host
    # grab the metadata which is in the top left corner
    metadata = list(asheet)[0]
    if 'site' in metadata:
        pass
#         print(metadata)
#     asheet.loc[:,"metadata"] = metadata
    asheet = asheet[list(asheet)[1:]]
    ion = asheet['Ion']
    # drop the comments section
    asheet.drop(['Comments'],axis=1,inplace=True)
    # drop the first column
#     asheet.drop(asheet.columns[0],axis=1,inplace=True)
    asheet.reset_index(drop=True,inplace=True)
    # drop the types column
    asheet.drop(['Types'],axis=1,inplace=True)
    # normalize the headers
    new_headers = {head:head.replace('mono','Mono').replace('total','Total').replace('dipo','Dipo').replace('self','Self') for head in list(asheet)}
    asheet.rename(columns=new_headers,inplace=True)
    dframes.append(asheet)
    if listin(list(asheet), 'mono') or listin(list(asheet), 'total'):
        print(list(asheet))

['Ion', 'Charge', 'Site', 'Symmetry', 'nd^N', 'B_{4,0} (Monopole)', 'B_{4,4} (Monopole)', 'B_{4,0} (Total)', 'B_{4,4} (Total)', 'Host']
['Ion', 'Charge', 'Site', 'Symmetry', 'nd^N', 'B_{2,0} (Monopole)', 'B_{4,0} (Monopole)', 'B_{4,3} (Monopole)', 'B_{2,0} (Total)', 'B_{4,0} (Total)', 'B_{4,3} (Total)', 'Host']
['Ion', 'Charge', 'Site', 'Symmetry', 'nd^N', 'B_{4,0} (Monopole)', 'B_{4,0} (Total)', 'B_{4,4} (Monopole)', 'B_{4,4} (Total)', 'Host']
['Ion', 'Charge', 'Site', 'Symmetry', 'nd^N', 'B_{2,0} (Monopole)', 'B_{4,0} (Monopole)', 'B_{4,3} (Monopole)', 'B_{2,0} (Total)', 'B_{4,0} (Total)', 'B_{4,3} (Total)', 'Host']
['Ion', 'Charge', 'Site', 'Symmetry', 'nd^N', 'B_{2,0} (Monopole)', 'B_{4,0} (Monopole)', 'B_{4,3} (Monopole)', 'B_{2,0} (Total)', 'B_{4,0} (Total)', 'B_{4,3} (Total)', 'Host']
['Ion', 'Site', 'Symmetry', 'Charge', 'nd^N', 'B_{2,0} (Monopole)', 'B_{4,0} (Monopole)', 'B_{4,3} (Monopole)', 'B_{2,0} (Total)', 'B_{4,0} (Total)', 'B_{4,3} (Total)', 'Host']
['Ion', 'Site', 'Sym

In [16]:
tot_df = pd.concat(dframes)
tot_df.reset_index(drop=True,inplace=True)

In [17]:
tot_df

,Ion,Site,Symmetry,Charge,nd^N,F^{(2)},F^{(4)},"B_{2,0}","B_{4,0}","B_{4,3}",...,"B_{4,0} (Monopole)","B_{4,4} (Monopole)","B_{4,0} (Total)","B_{4,4} (Total)","B_{2,0} (Monopole)","B_{4,3} (Monopole)","B_{2,0} (Total)","B_{4,3} (Total)",\zeta,Ref
0,Ti,Al_1,C_{3i},3,3d1,NaN,NaN,2953.0,-26934.0,30280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V,Al_1,C_{3i},3,3d2,56732.0,36132.0,2895.0,-26148.0,29397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cr,Al_1,C_{3i},3,3d3,59983.0,38912.0,2832.0,-25363.0,28514.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mn,Al_1,C_{3i},3,3d4,63234.0,41688.0,2769.0,-24577.0,27631.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fe,Al_1,C_{3i},3,3d5,66485.0,44463.0,2709.0,-23792.0,26748.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Ir,Zr,D_{2d},4,5d5,72980.0,49004.0,-8733.0,741.1,7026.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4814.0,NaN
95,Pt,Zr,D_{2d},4,5d6,75122.0,50424.0,-8161.0,639.4,6062.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5404.0,NaN
96,Au,Zr,D_{2d},4,5d7,77448.0,51994.0,-7619.0,548.9,5204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6045.0,NaN
97,Hg,Zr,D_{2d},4,5d8,79808.0,53587.0,-7021.0,455.2,4315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6736.0,NaN


In [718]:
list(set(list(tot_df)))

['B_{4,0} (Total)',
 'B_{4,3}',
 'B_{4,4} (total)',
 'B_{4,3} (monopole)',
 'B_{2,0} (total)',
 'Ref',
 'Symmetry',
 'F^{(4)}',
 'B_{4,0} (Monopole)',
 'F^{(2)}',
 'B_{4,0} (total)',
 'Host',
 'B_{4,0} (monopole)',
 '\\zeta',
 'B_{4,0}',
 'B_{2,0} (Total)',
 'B_{4,3} (total)',
 'B_{4,4} (monopole)',
 'B_{2,0}',
 'B_{4,3} (Monopole)',
 'B_{4,3} (Total)',
 'Ion',
 'nd^N',
 'Site',
 'B_{2,0} (monopole)',
 'Charge',
 'B_{2,0} (Monopole)']

In [715]:
for idx, df in enumerate(dframes):
    if '3d^N' in list(df):
        print(theoparams_sheets[idx],list(df))

## (OLD) Calculate which charge states of transition metals would give you d^n configurations

In [228]:
n=1
# returns a list of all the transition metal ions [(atomic_number, charge_state, configuration, term)]
# whose ground states includes exactly n equivalent d-electrons
tms = {}
for row  in [4,5,6]:
    tms.update(dict(map(lambda x: (x,Atom(x)),element_groups['transition metals'][row])))

In [229]:
chunks = []
for tm in tms:
    nist_datum = tms[tm].nist_data
    gstates = nist_datum[nist_datum['Level (eV)'] == 0]
    for i in range(1,11):
        if i == 1:
            seed = 'd'
        else:
            seed = 'd%d' % i
        chunk = gstates[gstates['Configuration'].apply(lambda x: (('3' + seed) in x.split('.')) or (('4' + seed) in x.split('.')) or (('5' + seed) in x.split('.')))][["Element","Charge","Configuration","Term","J"]]
        chunk["d^n"] = i
        chunk["Atomic Number"] = tm
        chunks.append(chunk)
ddframe = pd.concat(chunks)

In [230]:
# plt.close('ptable')
# font_multiplier = 2
# fig, ax = plt.subplots(num='ptable',figsize=font_multiplier*0.9*np.array([5.5,2.5]))
# margin = 0.02
# n = 1
# annotations = {atom.atomic_number :  '%.0f pm ' % atom.atomic_radius for atom in tms.values() if not(math.isnan(atom.atomic_radius))}

# for atom_num in tms:
#     atom = tms[atom_num]
#     sector = ddframe[((ddframe['d^n'] == n) | (ddframe['d^n'] == 10-n)) & (ddframe['Atomic Number'] == atom_num) & (ddframe['Charge'] < 5)]
#     charges = ','.join(list(map(lambda x: str(x)+'+',sector['Charge'])))
#     configs = ','.join(list(sector['Configuration']))
#     terms = ','.join(list(sector['Term']))
#     mdata = '\n'.join([charges, configs,terms])
#     group_loc = int(atom.group)
#     row = -atom.period
#     plt.text(group_loc+0.85, row-0.1, charges.replace(',','\n'), ha='center', va='top', fontsize=3*font_multiplier)
#     plt.text(group_loc+0.15, row-margin-0.1, terms.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
#     plt.text(group_loc+0.5, row-margin-0.4, configs.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
#     plt.text(group_loc+0.5, row-0.15, atom.symbol,ha='center',va='top',fontsize=7*font_multiplier, weight='bold')
#     plt.text(group_loc+0.5, row-margin-0.07, atom.atomic_number,ha='center',va='top',fontsize=1.75*font_multiplier)
#     plt.plot([group_loc+margin, group_loc+1-margin, group_loc+1-margin, group_loc+margin, group_loc+margin],
#             [ row-margin, row-margin, row-1+margin, row-1+margin, row-margin],'k-',lw=0.5)
# ax.axis('off')
# ax.set_aspect('equal')
# plt.tight_layout()
# plt.show()

In [227]:
plt.close('ptable')
font_multiplier = 2
fig, axes = plt.subplots(nrows=5, num='ptable',figsize=font_multiplier*0.9*np.array([5.5,5*1.75]))
margin = 0.02
n = 5
annotations = {atom.atomic_number :  '%.0f pm ' % atom.atomic_radius for atom in tms.values() if not(math.isnan(atom.atomic_radius))}
for n in [1,2,3,4,5]:
    ax = axes[n-1]
    for atom_num in tms:
        atom = tms[atom_num]
        sector = ddframe[((ddframe['d^n'] == n) | (ddframe['d^n'] == 10-n)) & (ddframe['Atomic Number'] == atom_num) & (ddframe['Charge'] < 5)]
        charges = ','.join(list(map(lambda x: str(x)+'+',sector['Charge'])))
        configs = ','.join(list(sector['Configuration']))
        terms = ','.join(list(sector['Term'])).replace('a ','')
        mdata = '\n'.join([charges, configs,terms])
        group_loc = int(atom.group)
        row = -atom.period
        ax.text(group_loc+0.85, row-0.1, charges.replace(',','\n'), ha='center', va='top', fontsize=3*font_multiplier)
        ax.text(group_loc+0.15, row-0.1, terms.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
        ax.text(group_loc+0.5, row-margin-0.4, configs.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
        ax.text(group_loc+0.5, row-0.15, atom.symbol,ha='center',va='top',fontsize=7*font_multiplier, weight='bold')
        ax.text(group_loc+0.5, row-margin-0.07, atom.atomic_number,ha='center',va='top',fontsize=1.75*font_multiplier)
        ax.plot([group_loc+margin, group_loc+1-margin, group_loc+1-margin, group_loc+margin, group_loc+margin],
                [ row-margin, row-margin, row-1+margin, row-1+margin, row-margin],'k-',lw=0.5)
        ax.axis('off')
        ax.set_aspect('equal')
        if n == 5:
            title = 'd${}^%d$' % (n)
        else:
            title = 'd${}^%d$, d${}^%d$' % (n,10-n)
        ax.set_title(title)
plt.tight_layout()

# plt.savefig('/Users/juan/Google Drive/Zia Lab/Log/Graphs/dn.pdf')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …